# **Data Visualization**

## Objectives

Achieve business requirement no.1:

* Visually differentiate a healthy cherry leaf from one that is affected by powdery mildew.


## Inputs

As a reminder here the structure of where the test, train, validation images will be selected: 
```bash
. 
├── inputs 
│   └──cherryleaves_dataset 
│      └──cherry-leaves                                     
│           ├── test
│           │   ├── healthy
│           │   └── powdery_mildew                   
│           ├── train
│           │   ├── healthy
│           │   └── powdery_mildew          
│           └── validation
│               ├── healthy
│               └── powdery_mildew                 
└── ...
```

## Outputs



* Compute average image size from the train set. The convolutional neural network (CNN) will train on the test set, hence is fundamental that all input images have the same size. Input size determines the CNN architecture.
* Image shape embeddings pickle file.
* Plot mean and vairability of images per label.
* Plot to distinguish the contrast between uninfected and infected cell images.
* Generate code that answers busines requirement 1 and can be used to build an image monatage on the Streamlit dashboard.



## Comments

* The avarage dimension of the raw images is 256px width by 256px height

* Data is also visually arranged as requested by the client (Business Requirement #1)


---

## Import libraries

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
import joblib
sns.set_style('white')
from matplotlib.image import imread

## Set working directory

In [4]:
cwd= os.getcwd()
cwd

'/workspace/mildew-leaves-detection/jupyter_notebooks'

In [6]:
os.chdir('/workspace/mildew-leaves-detection')
print("You set a new current directory")

You set a new current directory


In [8]:
work_dir = os.getcwd()
print("New current directory:", work_dir) 

New current directory: /workspace/mildew-leaves-detection
